# YOLOv8 Installation

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os

# Access Google Drive Folder
os.chdir("gdrive/MyDrive")

In [3]:
# Go to Project folder
%cd Vehicle-Detection

/content/gdrive/MyDrive/Vehicle-Detection


In [4]:
# Install YOLOv8
%pip install ultralytics

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.228 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.2/78.2 GB disk)


# Run Model on Video

In [ ]:
import cv2
import random
import time
import argparse
import os
from ultralytics import YOLO

def get_name(file_path):
  name_idx = 0
  file_pos = (file_path).rfind('\\')

  if(file_pos == -1):
      file_pos = (file_path).rfind('/')

      if(file_pos == -1):
          file_pos = 0

  name_idx = file_pos + 1

  name = file_path[name_idx:]

  return name

def get_save_path(file_name, folder_name):
  path = "result"
  save_path = os.path.join(path, folder_name)

  exists = os.path.exists(save_path)

  if(not exists):
      os.makedirs(save_path)

  save_path = os.path.join(save_path, file_name)

  return save_path

def draw_box(img, result, class_list, colors, label_size) :
  # Get information from result
  xyxy = result.boxes.xyxy.numpy()
  confidence = result.boxes.conf.numpy()
  class_id = result.boxes.cls.numpy().astype(int)
  # Pack together for easy use
  sum_output = list(zip(class_id, confidence, xyxy))
  # Copy image, in case that we need original image for something
  out_image = img.copy()

  for run_output in sum_output :
    # Unpack
    label, con, box = run_output
    # Choose color
    box_color = colors[int(label)]
    text_color = (255,255,255)
    # Get Class Name
    label = class_list[int(label)]
    # Draw object box
    first_half_box = (int(box[0]),int(box[1]))
    second_half_box = (int(box[2]),int(box[3]))
    cv2.rectangle(out_image, first_half_box, second_half_box, box_color, 2)
    # Create text
    text_print = '{label} {con:.2f}'.format(label = label, con = con)
    # Locate text position
    text_location = (int(box[0]), int(box[1] - 10 ))
    # Get size and baseline
    labelSize, baseLine = cv2.getTextSize(text_print, cv2.FONT_HERSHEY_SIMPLEX, label_size, 1)

    # Draw text's background
    cv2.rectangle(out_image
                    , (int(box[0]), int(box[1] - labelSize[1] - 10 ))
                    , (int(box[0])+labelSize[0], int(box[1] + baseLine-10))
                    , box_color , cv2.FILLED)
    # Put text
    cv2.putText(out_image, text_print ,text_location
                , cv2.FONT_HERSHEY_SIMPLEX , label_size
                , text_color, 2, cv2.LINE_AA)

  return out_image

def draw_fps(avg_fps, combined_img):
  avg_fps_str = float("{:.2f}".format(avg_fps))

  cv2.rectangle(combined_img, (10,2), (280,50), (255,255,255), -1)
  cv2.putText(combined_img, "FPS: "+str(avg_fps_str), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), thickness=3)

  return combined_img

In [ ]:
def detection(source, model, folder_name, img_size, label_size=1):
  # Initialize video
  cap = cv2.VideoCapture(source)

  # Initialize YOLOv8 model
  model_path = model
  yolov8_detector = YOLO(model_path)

  # Class Name and Colors
  label_map = yolov8_detector.names
  COLORS = [[random.randint(0, 255) for _ in range(3)] for _ in label_map]

  # FPS Detection
  frame_count = 0
  total_fps = 0
  avg_fps = 0

  # FPS Video
  total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  frame_width = int(cap.get(3))
  frame_height = int(cap.get(4))

  video_frames = []

  while cap.isOpened():
    # Press key q to stop
    if cv2.waitKey(1) == ord('q'):
        break

    try:
        # Read frame from the video
        ret, frame = cap.read()
        if not ret:
            break
    except Exception as e:
        print(e)
        continue

    # # Start Time
    start = time.time()
    # Update object localizer
    results = yolov8_detector.predict(frame, imgsz=img_size, verbose=False)
    result = results[0].cpu()

    # Draw Detection Results
    combined_img = draw_box(frame, result, label_map, COLORS, label_size)

    end = time.time()
    # # End Time

    # Draw FPS
    frame_count += 1
    fps = 1 / (end - start)
    total_fps = total_fps + fps
    avg_fps = total_fps / frame_count

    combined_img = draw_fps(avg_fps, combined_img)

    # Append frame to array
    video_frames.append(combined_img)

    #
    print("(%2d / %2d) Frames Processed" % (frame_count, total_frames))

  print("\nCreate a Video:")

  # Get a file name
  file_name = get_name(source)
  # Get Save Path
  save_path = get_save_path(file_name, folder_name)
  # Create VideoWriter object.
  out = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'XVID'), int(avg_fps), (frame_width, frame_height))

  for frame in video_frames:
    out.write(frame)

  out.release()

  print("Video is saved in: "+save_path)

In [ ]:
# Original YOLOv8 Model
detection("/content/gdrive/MyDrive/Vehicle-Detection/inference/Aerial View.mp4", "/content/gdrive/MyDrive/Vehicle-Detection/runs/detect/yolov8_vehicle/weights/best.pt", "yolov8l-vehicle", 640)

WARNING ⚠️ NMS time limit 0.550s exceeded
( 1 / 832) Frames Processed
( 2 / 832) Frames Processed
( 3 / 832) Frames Processed
( 4 / 832) Frames Processed
( 5 / 832) Frames Processed
( 6 / 832) Frames Processed
( 7 / 832) Frames Processed
( 8 / 832) Frames Processed
( 9 / 832) Frames Processed
(10 / 832) Frames Processed
(11 / 832) Frames Processed
(12 / 832) Frames Processed
(13 / 832) Frames Processed
(14 / 832) Frames Processed
(15 / 832) Frames Processed
(16 / 832) Frames Processed
(17 / 832) Frames Processed
(18 / 832) Frames Processed
(19 / 832) Frames Processed
(20 / 832) Frames Processed
(21 / 832) Frames Processed
(22 / 832) Frames Processed
(23 / 832) Frames Processed
(24 / 832) Frames Processed
(25 / 832) Frames Processed
(26 / 832) Frames Processed
(27 / 832) Frames Processed
(28 / 832) Frames Processed
(29 / 832) Frames Processed
(30 / 832) Frames Processed
(31 / 832) Frames Processed
(32 / 832) Frames Processed
(33 / 832) Frames Processed
(34 / 832) Frames Processed
(35 / 

In [ ]:
# Modified YOLOv8 Model
detection("/content/gdrive/MyDrive/Vehicle-Detection/inference/Aerial View.mp4", "/content/gdrive/MyDrive/Vehicle-Detection/runs/detect/yolov8_vehicle_small/weights/best.pt", "yolov8l-vehicle-small", 640)

( 1 / 832) Frames Processed
( 2 / 832) Frames Processed
( 3 / 832) Frames Processed
( 4 / 832) Frames Processed
( 5 / 832) Frames Processed
( 6 / 832) Frames Processed
( 7 / 832) Frames Processed
( 8 / 832) Frames Processed
( 9 / 832) Frames Processed
(10 / 832) Frames Processed
(11 / 832) Frames Processed
(12 / 832) Frames Processed
(13 / 832) Frames Processed
(14 / 832) Frames Processed
(15 / 832) Frames Processed
(16 / 832) Frames Processed
(17 / 832) Frames Processed
(18 / 832) Frames Processed
(19 / 832) Frames Processed
(20 / 832) Frames Processed
(21 / 832) Frames Processed
(22 / 832) Frames Processed
(23 / 832) Frames Processed
(24 / 832) Frames Processed
(25 / 832) Frames Processed
(26 / 832) Frames Processed
(27 / 832) Frames Processed
(28 / 832) Frames Processed
(29 / 832) Frames Processed
(30 / 832) Frames Processed
(31 / 832) Frames Processed
(32 / 832) Frames Processed
(33 / 832) Frames Processed
(34 / 832) Frames Processed
(35 / 832) Frames Processed
(36 / 832) Frames Pr